# Build an Agent

## Define tools

In [13]:
pip install --quiet langchain-anthropic langgraph


[notice] A new release of pip is available: 23.0.1 -> 24.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [14]:
from dotenv import load_dotenv
load_dotenv()

True

In [15]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in SF")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.78, 'lon': -122.42, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1719414479, 'localtime': '2024-06-26 8:07'}, 'current': {'last_updated_epoch': 1719414000, 'last_updated': '2024-06-26 08:00', 'temp_c': 13.3, 'temp_f': 55.9, 'is_day': 1, 'condition': {'text': 'Overcast', 'icon': '//cdn.weatherapi.com/weather/64x64/day/122.png', 'code': 1009}, 'wind_mph': 10.5, 'wind_kph': 16.9, 'wind_degree': 280, 'wind_dir': 'W', 'pressure_mb': 1017.0, 'pressure_in': 30.04, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 93, 'cloud': 100, 'feelslike_c': 12.5, 'feelslike_f': 54.5, 'windchill_c': 12.2, 'windchill_f': 53.9, 'heatindex_c': 13.1, 'heatindex_f': 55.6, 'dewpoint_c': 10.4, 'dewpoint_f': 50.6, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 4.0, 'gust_mph': 15.0, 'gust_kph': 24.1}}"}, {'url': 'https://world-weather.info/forecast/usa/sa

## Using Language Models

In [16]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4")

In [17]:
model_with_tools = model.bind_tools(tools)

In [18]:
from langchain_core.messages import HumanMessage
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: Hello! How can I assist you today?
ToolCalls: []


In [19]:
response = model_with_tools.invoke([HumanMessage(content="What's the weather in SF?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_4ucPVxW5Bxn6zVFeuE6oMAym'}]


## Create the agent

In [20]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

## Run the agent

In [21]:
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

response["messages"]

[HumanMessage(content='hi!', id='cc332b6b-9eb4-476f-a3fb-bfb145e16a81'),
 AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 83, 'total_tokens': 93}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-0bf77d0f-ccb6-43d4-811e-10b1aaf33ce9-0', usage_metadata={'input_tokens': 83, 'output_tokens': 10, 'total_tokens': 93})]

In [22]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
)
response["messages"]

[HumanMessage(content='whats the weather in sf?', id='a5d23618-fe38-4739-b334-1ab3555e464c'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_hLBRKFIHIEwidGKii7o5SVFG', 'function': {'arguments': '{\n  "query": "current weather in San Francisco"\n}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 88, 'total_tokens': 111}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-4cd77f2b-a700-45af-91b9-6b1de93c4448-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_hLBRKFIHIEwidGKii7o5SVFG'}], usage_metadata={'input_tokens': 88, 'output_tokens': 23, 'total_tokens': 111}),
 ToolMessage(content='[{"url": "https://www.weatherapi.com/", "content": "{\'location\': {\'name\': \'San Francisco\', \'region\': \'California\', \'country\': \'United States of Am

## Streaming Messages

In [23]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_WQlkPkDcjdQyUz47PvJ8c8Df', 'function': {'arguments': '{\n  "query": "current weather in San Francisco"\n}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 88, 'total_tokens': 111}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-525e23e6-825d-4561-81cc-c0e2883dda2a-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_WQlkPkDcjdQyUz47PvJ8c8Df'}], usage_metadata={'input_tokens': 88, 'output_tokens': 23, 'total_tokens': 111})]}}
----
{'tools': {'messages': [ToolMessage(content='[{"url": "https://www.weatherapi.com/", "content": "{\'location\': {\'name\': \'San Francisco\', \'region\': \'California\', \'country\': \'United States of America\', \'lat\': 37.78, \'lon\': -122.42

## Streaming tokens

In [24]:
async for event in agent_executor.astream_events(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}, version="v1"
):
    kind = event["event"]
    if kind == "on_chain_start":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print(
                f"Starting agent: {event['name']} with input: {event['data'].get('input')}"
            )
    elif kind == "on_chain_end":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print()
            print("--")
            print(
                f"Done agent: {event['name']} with output: {event['data'].get('output')['output']}"
            )
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            # Empty content in the context of OpenAI means
            # that the model is asking for a tool to be invoked.
            # So we only print non-empty content
            print(content, end="|")
    elif kind == "on_tool_start":
        print("--")
        print(
            f"Starting tool: {event['name']} with inputs: {event['data'].get('input')}"
        )
    elif kind == "on_tool_end":
        print(f"Done tool: {event['name']}")
        print(f"Tool output was: {event['data'].get('output')}")
        print("--")

/home/vscode/.local/lib/python3.10/site-packages/langchain_core/_api/beta_decorator.py:87: LangChainBetaWarning: This API is in beta and may change in the future.
  warn_beta(


--
Starting tool: tavily_search_results_json with inputs: {'query': 'current weather in San Francisco'}
Done tool: tavily_search_results_json
Tool output was: [{'url': 'https://www.wunderground.com/hourly/us/ca/san-francisco/94117/date/2024-6-26', 'content': 'Current Weather for Popular Cities . San Francisco, CA 62 ° F Partly Cloudy; Manhattan, NY 82 ° F Fair; Schiller Park, IL (60176) 81 ° F Cloudy; Boston, MA 81 ° F Partly Cloudy; Houston, TX ...'}, {'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.78, 'lon': -122.42, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1719414790, 'localtime': '2024-06-26 8:13'}, 'current': {'last_updated_epoch': 1719414000, 'last_updated': '2024-06-26 08:00', 'temp_c': 13.3, 'temp_f': 55.9, 'is_day': 1, 'condition': {'text': 'Overcast', 'icon': '//cdn.weatherapi.com/weather/64x64/day/122.png', 'code': 1009}, 'wind_mph': 10.5, 'wind_kph': 

## Adding in memory

In [28]:
from langgraph.checkpoint.sqlite import SqliteSaver

memory = SqliteSaver.from_conn_string(":memory:")

agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="Hi, I'm from San Francisco, California.")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 91, 'total_tokens': 101}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-8eafbe46-c961-446f-acd4-273c7f207d8a-0', usage_metadata={'input_tokens': 91, 'output_tokens': 10, 'total_tokens': 101})]}}
----


In [29]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="Whats the weather like now?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_XqJUN6XiUFeF1UiCiI3Yp2sx', 'function': {'arguments': '{\n  "query": "current weather in San Francisco, California"\n}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 114, 'total_tokens': 139}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-f93deab4-634d-4314-89a4-c943e03ed672-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco, California'}, 'id': 'call_XqJUN6XiUFeF1UiCiI3Yp2sx'}], usage_metadata={'input_tokens': 114, 'output_tokens': 25, 'total_tokens': 139})]}}
----
{'tools': {'messages': [ToolMessage(content='[{"url": "https://world-weather.info/forecast/usa/san_francisco/june-2024/", "content": "Extended weather forecast in San Francisco. Hourly Week 10 days 14 days 30 days Year.

In [30]:
config = {"configurable": {"thread_id": "xyz123"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="Whats the weather like now?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content="Sorry, I can't provide real-time information such as current weather updates. It would be best to check a trusted weather website or app for the most accurate and up-to-date information.", response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 87, 'total_tokens': 125}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-bcb4c871-ab79-4087-94b1-9a75809008d2-0', usage_metadata={'input_tokens': 87, 'output_tokens': 38, 'total_tokens': 125})]}}
----
